In [1]:
! pip install keybert

     |████████████████████████████████| 79 kB 3.8 MB/s 
     |████████████████████████████████| 224 kB 31.6 MB/s 
     |████████████████████████████████| 51 kB 6.8 MB/s 
     |████████████████████████████████| 3.8 MB 40.4 MB/s 
     |████████████████████████████████| 1.2 MB 39.5 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
     |████████████████████████████████| 6.5 MB 36.1 MB/s 
     |████████████████████████████████| 895 kB 38.8 MB/s 
     |████████████████████████████████| 596 kB 46.2 MB/s 
  Created wheel for keybert: filename=keybert-0.5.1-py3-none-any.whl size=21332 sha256=37de4d7f759ecac5aa00a37d5842b3cf1a9bb423a7c0e3432fbfbcea3251d8fe
  Stored in directory: /root/.cache/pip/wheels/8e/95/c5/f5ceed2a9f9e80bc1a706a10a6fb03d726df7a3dd11800a58b
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.0-py3-none-any.whl size=120747 sha256=c2a447338b8ee91f687eb56f8eb1cf7b41c91d123d3ed2f47783dd7d0ab8becf
  Stored in directory: /root/.cache/pip/whee

In [2]:
! pip install sentence_transformers

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import nltk
from google.colab import drive
import re
import en_core_web_sm
from collections import defaultdict
from keybert import KeyBERT
from sentence_transformers import SentenceTransformer, util

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
nlp = spacy.load("en_core_web_sm")

In [6]:
symmetric_model = SentenceTransformer('all-mpnet-base-v2')
asymmetric_model =  SentenceTransformer('msmarco-MiniLM-L-6-v3')

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/430 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
#Reading the data 
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/raw_paragraphs_0803_to_1203.csv")

In [8]:
#industry = Fashion, jewelery, country = UK
data.head()

,Unnamed: 0,id,resourceLink,tags,_ts,paragraph
0,0,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,chain that promotes trust in the global jewell...
1,1,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,Jewellery has always played a special role in ...
2,2,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,It was founded in 2005 by 14 member organisati...
3,3,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,"Today our membership has grown to more than 1,..."
4,4,4cc1e994-51c4-5aa9-9a06-c29a2de5174c.json,https://www.responsiblejewellery.com/wp-conten...,"{'external': [], 'benefit': 'energy saving', '...",1646934908,Our actions support the industry to be even mo...


In [9]:
data.columns

Index(['Unnamed: 0', 'id', 'resourceLink', 'tags', '_ts', 'paragraph'], dtype='object')

In [10]:
data.shape

(175604, 6)

In [11]:
data.isnull().sum()

Unnamed: 0      0
id              0
resourceLink    0
tags            0
_ts             0
paragraph       0
dtype: int64

In [12]:
data.duplicated().sum()

0

In [13]:
data['paragraph'].duplicated().sum()

6344

In [ ]:
# def fashion_preprocessing(text):
#   text= re.sub(r'\n',"",text)
#   text= re.sub(r'http://www.\w+.org/','',text)
#   text= re.sub(r'Figure\s\d:','', text)
#   text= re.sub(r'https://\w+.\w+/\d+.\d+/\w\d+\W\w+','',text)
#   text = re.sub('https?://\S+|www\.\S+', '', text)
#   text= re.sub(r'\Wwww.\w+\W\w+\W','',text)
#   text= re.sub(r'www.\w+','',text)
#   regrex_pattern = re.compile(pattern = "["
#         u"\U0001F600-\U0001F64F"  # emoticons
#         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
#         u"\U0001F680-\U0001F6FF"  # transport & map symbols
#         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
#         u"\U00002500-\U00002BEF"  # chinese char
#         u"\U00002702-\U000027B0"
#         u"\U00002702-\U000027B0"
#         u"\U000024C2-\U0001F251"
#         u"\U0001f926-\U0001f937"
#         u"\U00010000-\U0010ffff"
#         u"\u2640-\u2642" 
#         u"\u2600-\u2B55"
#         u"\u200d"
#         u"\u23cf"
#         u"\u23e9"
#         u"\u231a"
#         u"\ufe0f"  # dingbats
#         u"\u3030"  # flags (iOS)
#                            "]+", flags = re.UNICODE)
#   text= regrex_pattern.sub('', text)
#   tokens = [token for token in nlp(text) if not (token.is_stop or token.like_num or token.is_punct or token.is_space or len(token)==1)]
#   text= " ".join(map(str,tokens))
#   return text

In [35]:
class para_relevancy(object):

  def __init__(self, data, column_name):
    """
    data -------------> DataFrame which has the paragraphs/content column
    column_name ------> Name of the column which includes the content whihc will be used to extract the keywords
               
    """
    #asseritng the column_name
    assert type(column_name) == str, "column_name must be s String"
    self.__dataframe = data
    #creatinng the list of paragraphs
    self.__content = self.__dataframe[column_name].tolist()
    print("\n Removing duplicates......")
    len_bpre = len(self.__content)
    #removing duplicates
    self.__content = list(set(self.__content))
    len_apre = len(self.__content)
    print("\n Dupllicates removed: Rows affected {}".format(len_bpre-len_apre))
    #preprocessing the paras
    self.preprocessed = list(map(self.__preprocessor, self.__content))
    print("\n Preprocessing has been completed")

  #preprocessing function
  def __preprocessor(self,text):
    text= re.sub(r'\n',"",text)
    text= re.sub(r'http://www.\w+.org/','',text)
    text= re.sub(r'Figure\s\d:','', text)
    text= re.sub(r'https://\w+.\w+/\d+.\d+/\w\d+\W\w+','',text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text= re.sub(r'\Wwww.\w+\W\w+\W','',text)
    text= re.sub(r'www.\w+','',text)
    regrex_pattern = re.compile(pattern = "["
          u"\U0001F600-\U0001F64F"  # emoticons
          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
          u"\U0001F680-\U0001F6FF"  # transport & map symbols
          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
          u"\U00002500-\U00002BEF"  # chinese char
          u"\U00002702-\U000027B0"
          u"\U00002702-\U000027B0"
          u"\U000024C2-\U0001F251"
          u"\U0001f926-\U0001f937"
          u"\U00010000-\U0010ffff"
          u"\u2640-\u2642" 
          u"\u2600-\u2B55"
          u"\u200d"
          u"\u23cf"
          u"\u23e9"
          u"\u231a"
          u"\ufe0f"  # dingbats
          u"\u3030"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    text= regrex_pattern.sub('', text)
    tokens = [token for token in nlp(text) if not (token.is_stop or token.like_num or token.is_punct or token.is_space or len(token)==1)]
    text= " ".join(map(str,tokens))
    return text

  def unigram_key_word_extraction(self, save = False):
      """
      This method extracts the most significant key word from the given paras.
      Dataframe will be saved with the keywords and probabilites if save is switch to TRUE
      Default path of the save:"/content/drive/MyDrive/Colab Notebooks/
      
      """

      key_model = KeyBERT()
      #creating a dictionary
      self.__unigram_data = defaultdict(list)
      #attaching oringinal unique paras
      self.__unigram_data['Original_para'] = self.__content
      #attaching preprocessed paras
      self.__unigram_data['Preprocessed_para'] = self.preprocessed
      kw_model = KeyBERT()
      #creating list of key words 
      self.unigram_keywords = []
      for index,content in enumerate(self.preprocessed):
        #by preprocessing some elements become empty
        #First check the element is empty or not
        if content:
          #if content is not null, unigrams are extarcted 
          key_words = key_model.extract_keywords(content, top_n=5)
          print(index,key_words)
          self.__unigram_data['key_word'].append(key_words[0][0])
          self.__unigram_data['probability'].append(key_words[0][1])
          self.unigram_keywords.append(key_words[0][0])
        else:
          #if values are empty, tagged as Nan
          print(index,"Empty value")
          self.__unigram_data['key_word'].append("Nan")
          self.__unigram_data['probability'].append("Nan")
          self.unigram_keywords.append("Nan")
      #dictionary to dataframe
      self.__unigramdf = pd.DataFrame(self.__unigram_data)

      if save:
        #saving the model
        path = "/content/drive/MyDrive/Colab Notebooks/unigram_keywords.csv"
        self.__unigramdf.to_csv(path)
        print("CSV file has been successfully saved.")
      
      #returning the dataframe and the list of  keywords
      return self.__unigramdf, self.unigram_keywords

    
  def ngram_key_word_extraction(self,n_grams = 2, save = False):

      """ 
      This method generates ngrams of keywords.
      n_grams ----> Number of keywprds to extract
                    Ex: n_grams = 2--- "Industrial Technology"
                    Ex: n_grams = 3--- "Industrial Technology Standards"
                    Must be 2 or 3
      save -------> Bool, if True, dataframe will be saved to the drive with ngram
                    keywords and their probability
      """

      assert type(n_grams) == int, "n_gram must be an integer"
      #only bigrams and trigrams are allowed
      assert (n_grams > 1) and (n_grams < 4), "n_gram must be greater than 1 less than 4"

      key_model = KeyBERT()
      self.__ngram_data = defaultdict(list)
      self.__ngram_data['Original_para'] = self.__content
      self.__ngram_data['Preprocessed_para'] = self.preprocessed
      #creating list of bigrams or trigrams
      self.ngram_keywords = []
      for index,content in enumerate(self.preprocessed):
        #by preprocessing some elements become empty
        #First check the element is empty or not
        if content:
          #if content has elements
          list_content = content.split()
          #check whether content has elements more than specified n_grams
          #other wise error will be thrown
          if (len(list_content) > n_grams):
            #if has enough elements, keywords are extracted
            key_words1 = key_model.extract_keywords(content, keyphrase_ngram_range=(n_grams,n_grams), top_n = 5)
            print(index,key_words1)
            self.__ngram_data['key_word'].append(key_words1[0][0])
            self.__ngram_data['probability'].append(key_words1[0][1])
            self.ngram_keywords.append(key_words1[0][0])
          else:
            #if elements are less than n_grams, tagged as NAN
            print(index,"Empty String")
            self.__ngram_data['key_word'].append("Nan")
            self.__ngram_data['probability'].append("Nan")
            self.ngram_keywords.append("Nan")

        else:
          #if content is empty, tagged as Nan
          print(index,"Empty String")
          self.__ngram_data['key_word'].append("Nan")
          self.__ngram_data['probability'].append("Nan")
          self.ngram_keywords.append("Nan")

      #creating the dataframe
      self.__ngramdf = pd.DataFrame(self.__ngram_data)

      if save:
        #saving the model
        path = f"/content/drive/MyDrive/Colab Notebooks/{n_grams}ngram_keywords.csv"
        self.__ngramdf.to_csv(path)
        print("CSV file has been successfully saved.")

      
      return self.__ngramdf, self.ngram_keywords

  def check_relevance(self,specific_keyword,threshold ,save = False):

    """
    This method check whther the the paragraphs are relevant with the givcen specific word or sentence.
    Symmetric and asymmetric semantic search is used here to get the probabilities and 

    specific_keyword ---> string: Can bea word or a small sentence
    threshold ----------> int or float: Threshold value where the relevancy is checked.
    
    """
    #asseritng the threshold value
    assert (type(threshold == int)) or (type(threshold) == float), "Threshold must be an integer or float"
    #creating the dictionary to create dataframe finally
    self.__keyword_search = defaultdict(list)
    self.__keyword_search['orginal paras'] = self.__content
    self.__keyword_search['preprocessed paras'] = self.preprocessed
    #symmetric and assymeteric search will be used based on the length of the content
    #embedding the specific keyword or sentences to symmetric embedding----->to vector space
    query_embedding_sym = symmetric_model.encode(specific_keyword, convert_to_tensor=True)
    #embedding the the specific keyword or sentence inot asymmateric embedding
    query_embedding_asym = asymmetric_model.encode(specific_keyword, convert_to_tensor=True)
    for index,content in enumerate(self.preprocessed):
      #check the content is null or not null
      if content:
        len_content = len(content.split())
        #if the length of the content is larger than 5
        #asymmetrci embedding used
        if (len_content > (len(specific_keyword.split())+2)):
          print(index, "Asymmetric semantic search")
          #transforming the content in to embeddings
          corpus_embedding = asymmetric_model.encode(content, convert_to_tensor = True)
          #getting the similarity score
          similarity_score = util.semantic_search(query_embedding_asym, corpus_embedding, top_k = 1)
          if (similarity_score[0][0]['score'] > threshold):
            #if the similarity score is greater then the threshold value, it is tagged as relevant
            self.__keyword_search['Relevancy'].append("Relevant")
            self.__keyword_search['Probability'].append(similarity_score[0][0]['score'])
          else:
            #if similarity score  is smaller than the threshold, tagged as non-relevant
            self.__keyword_search['Relevancy'].append("Not Relevant")
            self.__keyword_search['Probability'].append(similarity_score[0][0]['score'])
        else:
          #if the length is less than 5, symmetric embedding used
          print(index,"Symmetric semantice search")
          #embeddin the content
          corpus_embedding1 = symmetric_model.encode(content, convert_to_tensor = True)
          #checking similarity
          similarity_score1 = util.semantic_search(query_embedding_sym, corpus_embedding1, top_k = 4)
          if (similarity_score1[0][0]['score'] > threshold):
            #if the similarity score is greater then the threshold value, it is tagged as relevant
            self.__keyword_search['Relevancy'].append("Relevant")
            self.__keyword_search['Probability'].append(similarity_score1[0][0]['score'])
          else:
            #if similarity score  is smaller than the threshold, tagged as non-relevant
            self.__keyword_search['Relevancy'].append("Not Relevant")
            self.__keyword_search['Probability'].append(similarity_score1[0][0]['score'])
      else:
        #if the content is null, tagged as Nan
        print("Empty  value")
        self.__keyword_search['Relevancy'].append("Nan")
        self.__keyword_search['Probability'].append("Nan")


    self.__keyword_dataframe = pd.DataFrame(self.__keyword_search)

    if save:
      #saving the dataframe
      path = "/content/drive/MyDrive/Colab Notebooks/key_wordsearch.csv"
      self.__keyword_dataframe.to_csv(path)
      print("CSV file has been successfully saved.")


    return self.__keyword_dataframe

In [44]:
df1 = data.iloc[:1000,:]

In [45]:
p2 = para_relevancy(df1,'paragraph')


 Removing duplicates......

 Dupllicates removed: Rows affected 59

 Preprocessing has been completed


In [46]:
df,list1 = p2.unigram_key_word_extraction(save=True)

0 [('textile', 0.5097), ('production', 0.3103), ('kpisframework', 0.3084), ('dyeing', 0.3056), ('suppliers', 0.2977)]
1 [('refurbishments', 0.4826), ('expenditure', 0.4286), ('fiscal', 0.4221), ('store', 0.3478), ('investment', 0.3)]
2 [('amended', 0.4246), ('filing', 0.3906), ('securities', 0.3861), ('incorporates', 0.3422), ('sec', 0.3177)]
3 [('sales', 0.6909), ('cost', 0.6405)]
4 [('partnerships', 0.5213), ('stakeholders', 0.5131), ('reporting', 0.3467), ('support', 0.3445), ('progress', 0.3227)]
5 [('carolina', 0.8568), ('south', 0.7768)]
6 [('lending', 0.3728), ('customers', 0.3559), ('criteria', 0.3493), ('credit', 0.3462), ('allowances', 0.3391)]
7 [('gemstones', 0.4834), ('risks', 0.4164), ('corruption', 0.3369), ('jewelry', 0.3088), ('media', 0.2992)]
8 [('debt', 0.726), ('cash', 0.6175), ('net', 0.4707)]
9 [('income', 0.7733), ('net', 0.5493)]
10 [('suppliers', 0.4008), ('silver', 0.3865), ('certifications', 0.3594), ('supplier', 0.3498), ('certification', 0.3286)]
11 [('emi

In [39]:
df.head()

,Original_para,Preprocessed_para,key_word,probability
0,Mills are encouraged to obtain and maintain th...,Mills encouraged obtain maintain following ISO...,textile,0.5097
1,Catalyzing Transformational Partnerships betwe...,Catalyzing Transformational Partnerships Unite...,partnerships,0.5213
2,South Carolina . . . . . . . . . . . . . . . ....,South Carolina,carolina,0.8568
3,The colored gemstones industry has been in the...,colored gemstones industry media spotlight pas...,gemstones,0.4834
4,• Actively avoid any conflicts of interest or ...,Actively avoid conflicts interest exercise Ker...,suppliers,0.4008


In [40]:
df1, list2 = p2.ngram_key_word_extraction(save = True)

0 [('textile mills', 0.6166), ('textile production', 0.5962), ('textile industry', 0.5816), ('performance textile', 0.5559), ('impacts textile', 0.4888)]
1 [('transformational partnerships', 0.6103), ('partnerships united', 0.548), ('making stakeholders', 0.5109), ('supporting reporting', 0.491), ('progress supporting', 0.4876)]
2 Empty String
3 [('gemstones industry', 0.6064), ('colored gemstones', 0.5752), ('gemstones developed', 0.5454), ('gemstones supply', 0.4974), ('gemstones use', 0.4933)]
4 [('silver suppliers', 0.5743), ('silver supply', 0.4919), ('brands silver', 0.4887), ('mining silver', 0.4842), ('silver products', 0.484)]
5 [('co2 emissions', 0.62), ('fossil fuels', 0.6179), ('fuels coal', 0.5492), ('peatlands co2', 0.528), ('combustion fossil', 0.5194)]
6 [('merchandise systems', 0.614), ('driven merchandise', 0.5402), ('inlargest marketing', 0.5311), ('responsive demand', 0.4995), ('demand driven', 0.4975)]
7 [('kering brands', 0.4811), ('conflict minerals', 0.4722), ('

In [41]:
df1.head()

,Original_para,Preprocessed_para,key_word,probability
0,Mills are encouraged to obtain and maintain th...,Mills encouraged obtain maintain following ISO...,textile mills,0.6166
1,Catalyzing Transformational Partnerships betwe...,Catalyzing Transformational Partnerships Unite...,transformational partnerships,0.6103
2,South Carolina . . . . . . . . . . . . . . . ....,South Carolina,Nan,Nan
3,The colored gemstones industry has been in the...,colored gemstones industry media spotlight pas...,gemstones industry,0.6064
4,• Actively avoid any conflicts of interest or ...,Actively avoid conflicts interest exercise Ker...,silver suppliers,0.5743


In [42]:
df2, list3 = p2.ngram_key_word_extraction(n_grams=3,save = True)

0 [('performance textile mills', 0.6392), ('textile production including', 0.6229), ('environmental performance textile', 0.6178), ('textile mills shall', 0.6143), ('textile industry optimize', 0.5893)]
1 [('transformational partnerships united', 0.6424), ('catalyzing transformational partnerships', 0.5943), ('progress supporting reporting', 0.5563), ('partnerships united nations', 0.5399), ('assessing progress supporting', 0.5254)]
2 Empty String
3 [('colored gemstones industry', 0.7107), ('responsible colored gemstones', 0.6572), ('gemstones industry media', 0.6513), ('colored gemstones developed', 0.612), ('colored gemstones supply', 0.5914)]
4 [('silver supply chain', 0.5961), ('containing silver suppliers', 0.5753), ('silver suppliers know', 0.5678), ('products silver suppliers', 0.5672), ('silver suppliers provide', 0.5621)]
5 [('combustion fossil fuels', 0.6543), ('emissions caused combustion', 0.6119), ('co2 emissions caused', 0.6097), ('dioxide co2 emissions', 0.609), ('fossil

In [47]:
df2 = p2.check_relevance("sustainability", 0.3, save=True)

0 Asymmetric semantic search
1 Asymmetric semantic search
2 Asymmetric semantic search
3 Symmetric semantice search
4 Asymmetric semantic search
5 Symmetric semantice search
6 Asymmetric semantic search
7 Asymmetric semantic search
8 Symmetric semantice search
9 Symmetric semantice search
10 Asymmetric semantic search
11 Asymmetric semantic search
12 Symmetric semantice search
13 Asymmetric semantic search
14 Symmetric semantice search
15 Asymmetric semantic search
16 Asymmetric semantic search
17 Symmetric semantice search
18 Symmetric semantice search
19 Asymmetric semantic search
20 Asymmetric semantic search
21 Asymmetric semantic search
22 Asymmetric semantic search
23 Symmetric semantice search
24 Asymmetric semantic search
25 Asymmetric semantic search
26 Asymmetric semantic search
27 Symmetric semantice search
28 Asymmetric semantic search
29 Symmetric semantice search
30 Asymmetric semantic search
31 Symmetric semantice search
32 Asymmetric semantic search
33 Symmetric semanti

In [ ]:
df2.head()

,orginal paras,preprocessed paras,Relevancy,Probability
0,chain that promotes trust in the global jewell...,chain promotes trust global jewellery Responsi...,Not Relevant,0.231637
1,Jewellery has always played a special role in ...,Jewellery played special role people lives car...,Not Relevant,0.185249
2,It was founded in 2005 by 14 member organisati...,founded member organisations included ABN AMRO...,Not Relevant,0.091612
3,"Today our membership has grown to more than 1,...",Today membership grown organisations committed...,Not Relevant,0.012436
4,Our actions support the industry to be even mo...,actions support industry responsible strengthe...,Not Relevant,0.152047


In [ ]:
p3 = para_relevancy(df1)


 Preprocessing has been completed


In [ ]:
d1 = p3.check_relevance("energy saving", 0.2)

0 Asymmetric semantic search
1 Asymmetric semantic search
2 Asymmetric semantic search
3 Asymmetric semantic search
4 Asymmetric semantic search
5 Asymmetric semantic search
6 Asymmetric semantic search
7 Asymmetric semantic search
8 Asymmetric semantic search
9 Asymmetric semantic search
10 Asymmetric semantic search
11 Asymmetric semantic search
12 Asymmetric semantic search
13 Asymmetric semantic search
14 Asymmetric semantic search
15 Asymmetric semantic search
16 Asymmetric semantic search
17 Asymmetric semantic search
18 Asymmetric semantic search
19 Asymmetric semantic search
20 Asymmetric semantic search
21 Asymmetric semantic search
22 Asymmetric semantic search
23 Asymmetric semantic search
24 Asymmetric semantic search
25 Asymmetric semantic search
26 Asymmetric semantic search
27 Asymmetric semantic search
28 Asymmetric semantic search
29 Asymmetric semantic search
30 Asymmetric semantic search
31 Asymmetric semantic search
32 Asymmetric semantic search
33 Asymmetric semant

In [ ]:
d1.head()

,orginal paras,preprocessed paras,Relevancy,Probability
0,chain that promotes trust in the global jewell...,chain promotes trust global jewellery Responsi...,Not Relevant,0.049949
1,Jewellery has always played a special role in ...,Jewellery played special role people lives car...,Not Relevant,0.053529
2,It was founded in 2005 by 14 member organisati...,founded member organisations included ABN AMRO...,Not Relevant,0.012260
3,"Today our membership has grown to more than 1,...",Today membership grown organisations committed...,Not Relevant,-0.054814
4,Our actions support the industry to be even mo...,actions support industry responsible strengthe...,Not Relevant,0.083860


In [ ]:
d1['Relevancy'].value_counts()

Not Relevant    957
Relevant         43
Name: Relevancy, dtype: int64